# Examine whether the predictive mobility ties are structurally important

- Computing the edge betweenness centrality of aggregate, strong, and weak mobility ties.
- Time cost: more than 2 hours due to iterations

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import copy
from scipy.sparse import csr_matrix


Bad key text.latex.unicode in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key text.latex.preview in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 125 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 157 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')


In [2]:
import time
from sklearn.preprocessing import normalize

In [3]:
import sys
sys.path.append("../")
import utils

In [4]:
import importlib
importlib.reload(utils)

<module 'utils' from '../utils.py'>

In [5]:
# read files
with open("../../data/02_intermediate/boston_stays.pickle", 'rb') as f:
    df_boston = pickle.load(f)

with open("../../data/02_intermediate/miami_stays.pickle", 'rb') as f:
    df_miami = pickle.load(f)

with open("../../data/02_intermediate/chicago_stays.pickle", 'rb') as f:
    df_chicago = pickle.load(f)
    

In [6]:
with open("../../data/03_processed/A_home_activity_three_cities_unweighted_dic.pickle", 'rb') as f:
    A_home_activity_unweighted_dic = pickle.load(f)
    
with open("../../data/03_processed/A_home_activity_three_cities_weighted_dic.pickle", 'rb') as f:
    A_home_activity_weighted_dic = pickle.load(f)


In [7]:
with open("../../data/03_processed/spatial_network_boston_miami_chicago_dic.pickle", 'rb') as f:
    spatial_network_dic = pickle.load(f)

In [8]:
# read shapefiles
with open("../../data/02_intermediate/boston_miami_chicago_ct_shp_dic.pickle", 'rb') as f:
    shp_dic = pickle.load(f)

In [9]:
# read lasso output from model_01
with open("../../data/05_model_outputs/lasso_coefficients.pickle", 'rb') as f:
    lasso_coef = pickle.load(f)

In [10]:
shp_dic.keys()

dict_keys(['boston', 'miami', 'chicago', 'total'])

In [11]:
shp_dic['chicago']

,STATEFP,COUNTYFP,TRACTCE,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
GEOID,,,,,,,,,,,,
17031010100,17,031,010100,101,Census Tract 101,G5020,S,379511,0,+42.0212553,-087.6698301,"POLYGON ((-87.67720 42.02294, -87.67628 42.022..."
17031010201,17,031,010201,102.01,Census Tract 102.01,G5020,S,504464,0,+42.0160077,-087.6801484,"POLYGON ((-87.68465 42.01948, -87.68432 42.019..."
17031010202,17,031,010202,102.02,Census Tract 102.02,G5020,S,351253,0,+42.0160476,-087.6733256,"POLYGON ((-87.67683 42.01941, -87.67674 42.019..."
17031010300,17,031,010300,103,Census Tract 103,G5020,S,472522,0,+42.0159431,-087.6665393,"POLYGON ((-87.67133 42.01937, -87.67121 42.019..."
17031010400,17,031,010400,104,Census Tract 104,G5020,S,548761,602605,+42.0064109,-087.6588157,"POLYGON ((-87.66345 42.01283, -87.66321 42.012..."
...,...,...,...,...,...,...,...,...,...,...,...,...
17197884005,17,197,884005,8840.05,Census Tract 8840.05,G5020,S,66773507,1770906,+41.2266861,-088.1332343,"POLYGON ((-88.18970 41.28982, -88.18945 41.289..."
17197884006,17,197,884006,8840.06,Census Tract 8840.06,G5020,S,72144850,1549117,+41.2556502,-088.0626126,"POLYGON ((-88.15386 41.29069, -88.15214 41.290..."
17197884101,17,197,884101,8841.01,Census Tract 8841.01,G5020,S,20893889,1121207,+41.6088678,-088.0959389,"POLYGON ((-88.14401 41.60929, -88.14345 41.609..."


### Preprocessing

#### 1. Compute the activity counts 

In [12]:
# activity counts for the three cities
# boston
activity_counts_boston = np.unique(df_boston.cat, return_counts = True)

# miami
activity_counts_miami = np.unique(df_miami.cat, return_counts = True)

# chicago
activity_counts_chicago = np.unique(df_chicago.cat, return_counts = True)

# convert the counts to df
activity_counts_dic = {}
activity_counts_dic['boston']=activity_counts_boston
activity_counts_dic['miami']=activity_counts_miami
activity_counts_dic['chicago']=activity_counts_chicago

# turn them to dataframes
activity_counts_df_dic = {}
for key_ in activity_counts_dic.keys():
    activity_counts = activity_counts_dic[key_]
    activity_count_df = pd.DataFrame(activity_counts[1],
                                 index = activity_counts[0],
                                 columns = ['count'])
    sorted_activity_count_df = activity_count_df.sort_values('count', ascending=False)
    activity_counts_df_dic[key_] = sorted_activity_count_df


In [13]:
shp_dic['boston']

,STATEFP,COUNTYFP,TRACTCE,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
GEOID,,,,,,,,,,,,
25009201100,25,009,201100,2011,Census Tract 2011,G5020,S,2786306,3481257,+42.4282342,-070.9260711,"POLYGON ((-70.94328 42.43815, -70.94324 42.438..."
25009202101,25,009,202101,2021.01,Census Tract 2021.01,G5020,S,1454942,7226,+42.4758943,-070.9085318,"POLYGON ((-70.92295 42.46878, -70.92292 42.468..."
25009202102,25,009,202102,2021.02,Census Tract 2021.02,G5020,S,2919051,24048,+42.4829911,-070.9199549,"POLYGON ((-70.93398 42.49080, -70.92844 42.489..."
25009202200,25,009,202200,2022,Census Tract 2022,G5020,S,3452798,1441782,+42.4695768,-070.8975175,"POLYGON ((-70.92226 42.46741, -70.92180 42.467..."
25009203100,25,009,203100,2031,Census Tract 2031,G5020,S,3720285,1481336,+42.4871332,-070.8724094,"POLYGON ((-70.89847 42.48483, -70.89828 42.484..."
...,...,...,...,...,...,...,...,...,...,...,...,...
33017085000,33,017,085000,850,Census Tract 850,G5020,S,120890933,5035519,+43.2155657,-071.0389797,"POLYGON ((-71.13352 43.20372, -71.13341 43.203..."
33017086000,33,017,086000,860,Census Tract 860,G5020,S,126932596,5722104,+43.2753082,-071.1700041,"POLYGON ((-71.24732 43.27528, -71.24721 43.275..."
33017087000,33,017,087000,870,Census Tract 870,G5020,S,94758837,909784,+43.3669235,-071.0625029,"POLYGON ((-71.15923 43.35524, -71.15892 43.355..."


#### 2. Aggregating the mobility networks

In [14]:
# initialize this dictionary
aggregate_mobility_network_dic = {}
threshold = 1.0

for city in ['boston', 'chicago', 'miami']:
    print(city)
    base_aggregate_mobility_network = pd.DataFrame(np.zeros((shp_dic[city].shape[0], shp_dic[city].shape[0])),
                                                  index = shp_dic[city].index, 
                                                  columns = shp_dic[city].index)
    base_aggregate_mobility_network.columns.name = 'GEOID'
    for activity_name in A_home_activity_unweighted_dic[city][threshold].keys():
        base_aggregate_mobility_network = base_aggregate_mobility_network.add(A_home_activity_unweighted_dic[city][threshold][activity_name], fill_value = 0.0)
    aggregate_mobility_network = utils.turn_df_to_adj(base_aggregate_mobility_network, shp_dic[city])
    aggregate_mobility_network[aggregate_mobility_network.values > 0.5] = 1.0
    aggregate_mobility_network_dic[city]=aggregate_mobility_network


boston
chicago
miami


In [15]:
aggregate_mobility_network_dic['boston']

GEOID,25009201100,25009202101,25009202102,25009202200,25009203100,25009203200,25009203301,25009203302,25009204101,25009204102,...,33017084200,33017084300,33017084400,33017084500,33017084600,33017085000,33017086000,33017087000,33017088000,33017088500
GEOID,,,,,,,,,,,,,,,,,,,,,
25009201100,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25009202101,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
25009202102,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
25009202200,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25009203100,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33017085000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
33017086000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
33017087000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0


#### 3. Creating a list of predictive mobility ties

In [16]:
### 
predictive_mobility_ties_three_cities = {}

city_list = ['boston', 'chicago', 'miami']
output_list = ['inc_median_household_2018','property_value_median_2018','rent_median_2018']
model_type = 'lasso (no socio-demographics)'
weak_mobility_tie_list = set()

for city in city_list:
    for output in output_list:
        weak_mobility_tie_list = weak_mobility_tie_list.union(set(lasso_coef[city][output][model_type].index))
    predictive_mobility_ties_three_cities[city] = list(weak_mobility_tie_list)

predictive_mobility_ties_three_cities


{'boston': ['Tennis Court',
  'University',
  'Fishing Store',
  'Noodles',
  'French',
  'Discount Store',
  'Hot Dogs',
  'Gastropub',
  'Locksmith',
  'Cycle Studio',
  'Caribbean',
  'Art Museum',
  'Synagogue',
  'Pet Service',
  'Spanish',
  'Fried Chicken',
  'Ski Area',
  'Brazilian',
  'Latin American',
  'Science Museum',
  'Video Games',
  'Bagels',
  'Food Stand',
  'Strip Club',
  'Indian Chinese',
  'Salad',
  'Skating Rink',
  'Football',
  'Storage',
  'Car Washes',
  'Laundromat',
  'Sorority House',
  'Engineering',
  'Thrift / Vintage',
  'Cupcakes',
  'Tattoo',
  'Convenience Store',
  'Peking Duck'],
 'chicago': ['Tennis Court',
  'Golf Course',
  'University',
  'Fishing Store',
  'Noodles',
  'Pawn Shop',
  'French',
  'Discount Store',
  'Curling Ice',
  'Hot Dogs',
  'Gastropub',
  'Pilates Studio',
  'Kitchen Supply',
  'Locksmith',
  'Cycle Studio',
  'Kosher',
  'Caribbean',
  'Art Museum',
  'Synagogue',
  'Gourmet',
  'North Indian',
  'Pet Service',
  'Sp

### Compute edge bet centralities for three graphs (each iteration)


In [17]:
shp_dic[city]

,STATEFP,COUNTYFP,TRACTCE,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
GEOID,,,,,,,,,,,,
12011010102,12,011,010102,101.02,Census Tract 101.02,G5020,S,1631521,53772,+26.2978686,-080.0882411,"POLYGON ((-80.09511 26.29210, -80.09489 26.293..."
12011010103,12,011,010103,101.03,Census Tract 101.03,G5020,S,2089489,193668,+26.3117373,-080.0863071,"POLYGON ((-80.09291 26.30531, -80.09280 26.305..."
12011010104,12,011,010104,101.04,Census Tract 101.04,G5020,S,803528,806760,+26.3131776,-080.0768224,"POLYGON ((-80.08121 26.31567, -80.08119 26.315..."
12011010304,12,011,010304,103.04,Census Tract 103.04,G5020,S,1196583,0,+26.3008841,-080.1137229,"POLYGON ((-80.12188 26.29627, -80.12168 26.296..."
12011010305,12,011,010305,103.05,Census Tract 103.05,G5020,S,2408192,49044,+26.3107830,-080.1140738,"POLYGON ((-80.12636 26.30452, -80.12558 26.306..."
...,...,...,...,...,...,...,...,...,...,...,...,...
12099008201,12,099,008201,82.01,Census Tract 82.01,G5020,S,1173006,0,+26.6867976,-080.6785674,"POLYGON ((-80.68404 26.68256, -80.68401 26.682..."
12099008202,12,099,008202,82.02,Census Tract 82.02,G5020,S,925852,0,+26.6812553,-080.6755362,"POLYGON ((-80.68644 26.67826, -80.68639 26.678..."
12099008203,12,099,008203,82.03,Census Tract 82.03,G5020,S,47929650,0,+26.6458764,-080.6366385,"POLYGON ((-80.68695 26.66398, -80.68695 26.664..."


In [18]:
A_home_activity_unweighted_dic[city][threshold].keys()

dict_keys(['ATM', 'Academic Building', 'Accessories', 'Acupuncturist', 'Administrative Building', 'Adult Boutique', 'Adult Education Center', 'Advertising Agency', 'Afghan', 'African', 'Airport', 'Airport Service', 'Alternative Healer', 'American', 'Amphitheater', 'Animal Shelter', 'Antiques', 'Apparel', 'Apres Ski Bar', 'Aquarium', 'Arcade', 'Arepas', 'Argentinian', 'Art Gallery', 'Art Museum', 'Art Studio', 'Arts', 'Arts & Crafts', 'Arts & Entertainment', 'Asian', 'Assisted Living', 'Astrologer', 'Athletics & Sports', 'Auditorium', 'Australian', 'Auto Dealer', 'Auto Garage', 'Auto Workshop', 'Automotive', 'B & B', 'BBQ', 'Baby Store', 'Bagels', 'Baggage Claim', 'Bakery', 'Ballroom', 'Bank', 'Bar', 'Baseball', 'Baseball Field', 'Basketball', 'Basketball Court', 'Bath House', 'Bathing Area', 'Bay', 'Beach', 'Beach Bar', 'Beer Bar', 'Beer Garden', 'Beer Store', 'Belgian', 'Big Box Store', 'Bike', 'Bike Shop', 'Bike Trail', 'Billiards', 'Bistro', 'Board Shop', 'Boarding', 'Boat / Ferry',

In [19]:
predictive_mobility_ties_three_cities[city]

['Perfume Shop',
 'Baseball',
 'Cantonese',
 'Golf Course',
 'University',
 'French',
 'Discount Store',
 'Kitchen Supply',
 'Pilates Studio',
 'Tennis',
 'Caribbean',
 'Art Museum',
 'North Indian',
 'Spanish',
 'Fried Chicken',
 'Gay Bar',
 'Bistro',
 'Fast Food',
 'Science Museum',
 'Basketball',
 'Smoke Shop',
 'Big Box Store',
 'Sorority House',
 'Grilled Meat Restaurants',
 'Video Games',
 'Food',
 'Bagels',
 'Strip Club',
 'Modern European',
 'Indian Chinese',
 'Theater',
 'Skating Rink',
 'Football',
 'Concert Hall',
 'New American',
 'Laundromat',
 'Religious School',
 'Hockey',
 'Cupcakes',
 'Convenience Store',
 'Tennis Court',
 'Fishing Store',
 'Noodles',
 'Pawn Shop',
 'Sushi',
 'Curling Ice',
 'Hot Dogs',
 'Gastropub',
 'Locksmith',
 'Cycle Studio',
 'Gourmet',
 'Synagogue',
 'Pet Service',
 'Circus',
 'Check Cashing Service',
 'Recreation Center',
 'Ski Area',
 'Buffet',
 'Casino',
 'Supermarket',
 'Soccer Field',
 'Lighthouse',
 'Brazilian',
 'Latin American',
 'Climbi

In [20]:
# init
network_edge_bet_centrality_dic = {}

Unclear why I could encounter a key error in the following sampling - Peking Duck exists in the lasso regression results, but not the unweighted graph.

In [21]:
# Need a function for sampling N edges from an adjacency network. 
# Using spatial network as the base. Then augment (1) aggregate network, (2) strong network, and (3) weak network. 
# Use each weak mobility network to create a benchmark. Randomly sample

# time
beginning_time = time.time()

# fixed parameters
spatial_net_name = 'queen_contiguity_adj_df'
model_type = 'lasso (no socio-demographics)'
threshold = 1.0
top_K_as_strong_mobility_ties = 50 # arbitrary. 

for city in ['boston','miami','chicago']:
# for city in ['miami']:
    # city = 'boston'
    print(city)
    network_edge_bet_centrality_dic[city] = {}

    ###
    spatial_weak_edge_bet_centrality_list = []
    weak_edge_bet_centrality_list = []
    spatial_agg_edge_bet_centrality_list = []
    agg_edge_bet_centrality_list = []
    spatial_strong_edge_bet_centrality_list = []
    strong_edge_bet_centrality_list = []
    edge_count_list = []
    weak_mobility_tie_names = []
    strong_mobility_tie_names = []

    # base spatial network
    spatial_net = spatial_network_dic[city][spatial_net_name]
    G_spatial = nx.from_pandas_adjacency(spatial_net)

    if nx.number_connected_components(G_spatial) > 1:
        # if city = chicago or miami, then the network is disconnected. We choose the giant component. 
        # find the giant component
        Gcc = sorted(nx.connected_components(G_spatial), key=len, reverse=True)
        for G_sub in Gcc:
            print(len(G_sub)) # print the size of the components.
        G0 = G_spatial.subgraph(Gcc[0])
        giant_component_node_list = sorted(list(G0.nodes))
        giant_component_node_list

        ## replace the input shapefile and the spatial networks!
        spatial_network_dic[city][spatial_net_name] = spatial_network_dic[city][spatial_net_name].loc[giant_component_node_list, giant_component_node_list]
        shp_dic[city] = shp_dic[city].loc[giant_component_node_list, :]
        aggregate_mobility_network_dic[city] = aggregate_mobility_network_dic[city].loc[giant_component_node_list, giant_component_node_list]

        # recreate the spatial net and spatial graph
        spatial_net = spatial_network_dic[city][spatial_net_name]
        G_spatial = nx.from_pandas_adjacency(spatial_net)

        
    # computing edge bet centralities
    for idx in range(len(predictive_mobility_ties_three_cities[city])):
        weak_activity_name = predictive_mobility_ties_three_cities[city][idx]

        # timing
        current_time = time.time()
        elapse_time = current_time - beginning_time
        print(idx, weak_activity_name, elapse_time/60.0, "minutes", end = '\r')

        try:
            # for some unknown reason, the predictive mobility ties may not exist in A_home_activity_unweighted_dic. 
            # weak    
            weak_mobility_net = utils.turn_df_to_adj(A_home_activity_unweighted_dic[city][threshold][weak_activity_name], shp_dic[city])
            weak_tie_total = weak_mobility_net.sum().sum() # weak tie total counts is the benchmark.

            # strong ratio initialization
            sampling_ratio_from_strong = 2.0 # initialize a value larger than 1.0
            while sampling_ratio_from_strong > 1.0:
                # strong ratio should be smaller than 1.0
                # otherwise sampling a strong tie again.
                strong_activity_name = np.random.choice(activity_counts_df_dic[city].index[:top_K_as_strong_mobility_ties])
                strong_mobility_net = utils.turn_df_to_adj(A_home_activity_unweighted_dic[city][threshold][strong_activity_name], shp_dic[city])
                strong_tie_total = strong_mobility_net.sum().sum()
                sampling_ratio_from_strong = weak_tie_total/strong_tie_total

            # aggregate network
            aggregate_mobility_network = aggregate_mobility_network_dic[city]
            aggregate_tie_total = aggregate_mobility_network.sum().sum()
            # agg ratio 
            sampling_ratio_from_aggregae = weak_tie_total/aggregate_tie_total

            # sampling the strong and aggregate mobility networks
            mask_matrix_for_strong = np.random.choice([0,1], size = (spatial_net.shape), p = [1-sampling_ratio_from_strong, sampling_ratio_from_strong])
            sampled_strong_mobility_net = strong_mobility_net * mask_matrix_for_strong
            mask_matrix_for_aggregate = np.random.choice([0,1], size = (spatial_net.shape), p = [1-sampling_ratio_from_aggregae, sampling_ratio_from_aggregae])
            sampled_aggregate_mobility_net = aggregate_mobility_network * mask_matrix_for_aggregate

            # Creating three integrated adjacency matrix. Computing the 
            # 1. integrating weak and spatial
            integrated_weak_adj = spatial_net.add(weak_mobility_net, fill_value = 0.0)
            integrated_weak_adj.values[integrated_weak_adj.values > 1.0] = 1.0
            G_integrated_weak = nx.from_pandas_adjacency(integrated_weak_adj)
            edge_bet_centrality_integerated_weak_graph = nx.edge_betweenness_centrality(G_integrated_weak)
            edge_bet_centrality_integerated_weak_df = pd.DataFrame(edge_bet_centrality_integerated_weak_graph.values(), 
                                                                   index = list(edge_bet_centrality_integerated_weak_graph.keys()), 
                                                                   columns = ['edge_bet_centrality'])
            G_spatial = nx.from_pandas_adjacency(spatial_net)
            G_weak_mobility = nx.from_pandas_adjacency(weak_mobility_net)
            spatial_edges = list(G_spatial.edges())
            weak_mobility_edges = list(G_weak_mobility.edges())
            # save
            spatial_weak_edge_bet_centrality_list.append(edge_bet_centrality_integerated_weak_df.loc[spatial_edges,].mean().values[0])
            weak_edge_bet_centrality_list.append(edge_bet_centrality_integerated_weak_df.loc[weak_mobility_edges,].mean().values[0])

            # 2. integrating strong and spatial
            integrated_strong_adj = spatial_net.add(sampled_strong_mobility_net, fill_value = 0.0)
            integrated_strong_adj.values[integrated_strong_adj.values > 1.0] = 1.0
            G_integrated_strong = nx.from_pandas_adjacency(integrated_strong_adj)
            edge_bet_centrality_integerated_strong_graph = nx.edge_betweenness_centrality(G_integrated_strong)
            edge_bet_centrality_integerated_strong_df = pd.DataFrame(edge_bet_centrality_integerated_strong_graph.values(), 
                                                                   index = list(edge_bet_centrality_integerated_strong_graph.keys()), 
                                                                   columns = ['edge_bet_centrality'])
            G_spatial = nx.from_pandas_adjacency(spatial_net)
            G_strong_mobility = nx.from_pandas_adjacency(sampled_strong_mobility_net)
            spatial_edges = list(G_spatial.edges())
            strong_mobility_edges = list(G_strong_mobility.edges())
            # save
            spatial_strong_edge_bet_centrality_list.append(edge_bet_centrality_integerated_strong_df.loc[spatial_edges,].mean().values[0])
            strong_edge_bet_centrality_list.append(edge_bet_centrality_integerated_strong_df.loc[strong_mobility_edges,].mean().values[0])

            # 3. integrating aggregate and spatial
            integrated_agg_adj = spatial_net.add(sampled_aggregate_mobility_net, fill_value = 0.0)
            integrated_agg_adj.values[integrated_agg_adj.values > 1.0] = 1.0
            G_integrated_agg = nx.from_pandas_adjacency(integrated_agg_adj)
            edge_bet_centrality_integerated_agg_graph = nx.edge_betweenness_centrality(G_integrated_agg)
            edge_bet_centrality_integerated_agg_df = pd.DataFrame(edge_bet_centrality_integerated_agg_graph.values(), 
                                                                   index = list(edge_bet_centrality_integerated_agg_graph.keys()), 
                                                                   columns = ['edge_bet_centrality'])
            G_spatial = nx.from_pandas_adjacency(spatial_net)
            G_agg_mobility = nx.from_pandas_adjacency(sampled_aggregate_mobility_net)
            spatial_edges = list(G_spatial.edges())
            agg_mobility_edges = list(G_agg_mobility.edges())
            # save
            spatial_agg_edge_bet_centrality_list.append(edge_bet_centrality_integerated_agg_df.loc[spatial_edges,].mean().values[0])
            agg_edge_bet_centrality_list.append(edge_bet_centrality_integerated_agg_df.loc[agg_mobility_edges,].mean().values[0])

            # save other info
            edge_count_list.append(weak_tie_total)
            weak_mobility_tie_names.append(weak_activity_name)
            strong_mobility_tie_names.append(strong_activity_name)
            
        except KeyError:
            continue

    # save
    network_edge_bet_centrality_dic[city]['spatial_weak_edge_bet_centrality_list']=spatial_weak_edge_bet_centrality_list
    network_edge_bet_centrality_dic[city]['weak_edge_bet_centrality_list']=weak_edge_bet_centrality_list
    network_edge_bet_centrality_dic[city]['spatial_agg_edge_bet_centrality_list']=spatial_agg_edge_bet_centrality_list
    network_edge_bet_centrality_dic[city]['agg_edge_bet_centrality_list']=agg_edge_bet_centrality_list
    network_edge_bet_centrality_dic[city]['spatial_strong_edge_bet_centrality_list']=spatial_strong_edge_bet_centrality_list
    network_edge_bet_centrality_dic[city]['strong_edge_bet_centrality_list']=strong_edge_bet_centrality_list
    network_edge_bet_centrality_dic[city]['edge_count_list']=edge_count_list
    network_edge_bet_centrality_dic[city]['weak_mobility_tie_names']=weak_mobility_tie_names
    network_edge_bet_centrality_dic[city]['strong_mobility_tie_names']=strong_mobility_tie_names


boston


KeyboardInterrupt: 

In [ ]:
network_edge_bet_centrality_dic['miami']

In [ ]:
network_edge_bet_centrality_dic['chicago']

# Save

In [36]:
with open('../../data/05_model_outputs/network_new_edge_bet_centrality_dic.pickle', 'wb') as f:
    pickle.dump(network_edge_bet_centrality_dic, f)
